In [1]:
import bpy
import json
from math import *

In [2]:
# bpy 초기화
# clear scene
for elem in bpy.data.objects:
    bpy.data.objects.remove(bpy.data.objects[elem.name], do_unlink=True)
# fbx 메인 모델 불러오기
bpy.ops.import_scene.fbx(
    filepath="default2.fbx"
)

ob = bpy.data.objects['Armature']
bpy.ops.object.mode_set(mode='POSE')

FBX Import: start importing default2.fbx
FBX version: 7400
	FBX import: Prepare...
		Done (0.000000 sec)

	FBX import: Templates...
		Done (0.000000 sec)

	FBX import: Nodes...
		Done (0.000000 sec)

	FBX import: Connections...
		Done (0.000000 sec)

	FBX import: Meshes...
		Done (0.421875 sec)

	FBX import: Materials & Textures...
		Done (0.031250 sec)

	FBX import: Cameras & Lamps...
		Done (0.000000 sec)

	FBX import: Objects & Armatures...
		Done (0.031250 sec)

	FBX import: ShapeKeys...
		Done (0.000000 sec)

	FBX import: Animations...
		Done (0.031250 sec)

	FBX import: Assign materials...
		Done (0.000000 sec)

	FBX import: Assign textures...
		Done (0.000000 sec)

	FBX import: Cycles z-offset workaround...
		Done (0.000000 sec)

	Done (0.625000 sec)

Import finished.


{'FINISHED'}

# 3d 모션 제작 함수

In [3]:
# Func 관절 기초값 설정
def defaultSetting(part, facing, angle):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'
    pbone.rotation_euler.rotate_axis(facing, angle)
# Func 윗팔 각도 조정
def UpperArmX(part, rotation, frame):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'
    pbone.rotation_euler.rotate_axis("X", -rotation)
    pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
    pbone.rotation_euler.rotate_axis("X", rotation)

In [4]:
# Func 손가락 각도 조정
def fingerRotate(part, rotation, frame):
    pbone = ob.pose.bones[part]
    pbone.rotation_mode = 'XYZ'
    if "Thumb" in part:
        weight = 1
        if "Left" in part:
            weight = -1
        pbone.rotation_euler.rotate_axis("Z", -rotation * weight)
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("Z", rotation * weight)
    else:
        pbone.rotation_euler.rotate_axis("X", -rotation)
        pbone.keyframe_insert(data_path="rotation_euler", frame = frame)
        pbone.rotation_euler.rotate_axis("X", rotation)

In [5]:
# 팔 기본 각도 세팅
defaultSetting("LeftArm", "X", radians(90))
defaultSetting("RightArm", "X", radians(90))

In [6]:
# json을 통한 모델 리깅
with open('./test.json', 'r') as f:
    frames = json.load(f)
last_frame = int([*frames.keys()][-1].replace("frame_", ""))
for fps in range(last_frame):
    # 어깨 - 팔 각도 조정
    UpperArmX("LeftArm", frames[f"frame_{fps}"]["left"]["upperArm"], fps)
    UpperArmX("RightArm", frames[f"frame_{fps}"]["right"]["upperArm"], fps)
    # 손가락 각도 조정
    for name in ["Thumb", "Index", "Middle", "Ring", "Pinky"]:
        for idx in range(1, 4):
            if "hand" in frames[f"frame_{fps}"]["left"]:
                fingerRotate(f"LeftHand{name}{idx}", -frames[f"frame_{fps}"]["left"]["hand"][name][str(idx)], fps)
            if "hand" in frames[f"frame_{fps}"]["right"]:
                fingerRotate(f"RightHand{name}{idx}", -frames[f"frame_{fps}"]["right"]["hand"][name][str(idx)], fps)

    print(f'{round(fps/last_frame, 3)  * 100}% 진행중...')
print("완료")

완료


In [ ]:
#fbx 저장
bpy.ops.export_scene.fbx(
    filepath=bpy.path.abspath("test.fbx"),
    use_active_collection=True
)